In [18]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import ttest_ind, ttest_rel, pearsonr
from sklearn import preprocessing

ModuleNotFoundError: No module named 'scipy'

In [ ]:
install scipy

SyntaxError: invalid syntax (1168798691.py, line 1)

In [ ]:
# Get data on Currency Changes from Yahoo Finance for 5 years
#currency_data = yf.Ticker("EURUSD=X")
#df_currency = currency_data.history(period="5y")

# Export the data to a csv file 
#path = '../Data/eur_usd_exchange_rate.csv'
#df_currency.to_csv(path)

(1306, 7)


In [4]:
currency_data = pd.read_csv('../Data/eur_usd_exchange_rate.csv')
currency_data

stock_data = pd.read_csv('../Data/loreal_stock_price.csv')
stock_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2019-10-30 00:00:00+01:00,240.157799,244.117473,237.119004,242.275772,1172925,0.0,0.0
1,2019-10-31 00:00:00+01:00,243.012451,245.498758,240.986596,241.078659,758477,0.0,0.0
2,2019-11-01 00:00:00+01:00,241.447007,243.104527,238.868609,239.789459,301658,0.0,0.0
3,2019-11-04 00:00:00+01:00,239.789455,241.447003,238.224026,239.881546,432258,0.0,0.0
4,2019-11-05 00:00:00+01:00,239.973657,242.367874,238.960716,242.367874,461773,0.0,0.0
...,...,...,...,...,...,...,...,...
1280,2024-10-24 00:00:00+02:00,359.750000,364.000000,357.799988,358.700012,360703,0.0,0.0
1281,2024-10-25 00:00:00+02:00,358.649994,358.799988,354.649994,355.649994,299447,0.0,0.0
1282,2024-10-28 00:00:00+01:00,358.799988,360.899994,356.649994,359.549988,317300,0.0,0.0
1283,2024-10-29 00:00:00+01:00,361.450012,362.399994,354.350006,355.600006,307733,0.0,0.0


In [ ]:
# Match data of stock prices and currency changes
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
currency_data['']